In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from datetime import datetime as dt
import re
import MeCab
import ipadic
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import scipy as sp
import collections

In [2]:
# xlsxファイルをdfに変換する
df = pd.read_excel("/app/data/patent_20211206.xlsx",header=0) 

In [3]:
df.columns

Index(['公開・公表番号', '発明の名称', '公開・公表日', '出願日', '原出願日', '出願人・権利者名', '出願番号', '登録番号',
       '筆頭発明者', '発明者名', 'IPC', '筆頭ＩＰＣ', 'Ｆターム', 'FI', 'ステイタス', 'ST',
       '早期請求出願有無', '要約', '全請求項', '詳細な説明'],
      dtype='object')

In [4]:
# drop_list = ['原出願日', '出願番号', '登録番号','ステイタス', 'ST','']
drop_list = ['公開・公表番号', '登録番号', 'ST', 'ステイタス', '発明者名', '発明の名称', '出願日', '原出願日', '筆頭発明者', '早期請求出願有無']

df = df.drop(drop_list, axis=1)

In [5]:
df.fillna('')

,公開・公表日,出願人・権利者名,出願番号,IPC,筆頭ＩＰＣ,Ｆターム,FI,要約,全請求項,詳細な説明
0,2001.12.14,佐藤工業株式会社,実願2001-003729,B65D 5/491 (2006.01),B65D 5/491 (2006.01),"3E060AA03,3E060AB05,3E060BC02,3E060CB04,3E060C...","B65D 5/48 101J,B65D 5/498",【要約】【課題】 強度が劣る原紙を用いたとしても箱の強度が維持される外装用段ボール箱を得んと...,【実用新案登録請求の範囲】【請求項１】 広幅と狭幅の各一対の側板を交互に横方向に連設し、それ...,【考案の詳細な説明】 【０００１】 【考案の属する技術分野】 本考案は、袋菓子または詰...
1,2001.11.16,"寄神建設株式会社,東亜建設工業株式会社,社団法人日本海上起重技術協会",実願2001-002834,B66C 1/36 (2006.01),B66C 1/36 (2006.01),"3F004CC01,3F004CD08",B66C 1/36 A,【要約】【課題】 重量物を吊り下げて設置を行う際の、重量物の開放を遠隔操作で容易かつ安全に行...,【実用新案登録請求の範囲】【請求項１】 一端がワイヤで吊り下げ支持される少なくとも一枚のフッ...,【考案の詳細な説明】【０００１】【考案の属する技術分野】 本考案は、クレーン等によりコンクリ...
2,2001.11.09,株式会社大林組,特願2001-120638,"E01D 19/02 (2006.01),E01D 21/00 (2006.01)",E01D 19/02 (2006.01),"2D059AA03,2D059CC04","E01D 19/02 ,E01D 21/04 ,E01D 21/00...",【要約】【目的】 作業足場や型枠の組立，解体が不要になる橋脚柱の構築工法の提供。【構成】 構...,【特許請求の範囲】【請求項１】 フーチング部と橋梁との間に上下端を一体化させて構築される橋脚...,【発明の詳細な説明】【０００１】【産業上の利用分野】この発明は、橋脚柱の構築工法および橋脚柱...
3,2001.12.26,株式会社大林組,特願2001-109004,"E02B 7/00 (2006.01),B65G 33/12 (2006.01)",E02B 7/00 (2006.01),"3F040BA01,3F040CA04,3F040EA03","E02B 7/00 C,B65G 33/12",【要約】【課題】 材料分離と運搬費用の低減。【解決手段】 シュート式打設装置１２は、円筒状の...,【特許請求の範囲】【請求項１】 回転自在に支持される両端が開口した円筒状のドラムと、前記ドラ...,【発明の詳細な説明】【０００１】【発明の属する技術分野】この発明は、コンクリートの運搬装置に...
4,2001.12.21,株式会社大林組,特願2001-108995,E02B 7/00 (2006.01),E02B 7/00 (2006.01),,E02B 7/00 C,【要約】【課題】 材料分離と運搬費用の低減。【解決手段】 シュート式打設装置１２は、円筒状の...,【特許請求の範囲】【請求項１】 斜面の上端側から下端側に向けてコンクリートを運搬する方法にお...,【発明の詳細な説明】【０００１】【発明の属する技術分野】この発明は、コンクリートの運搬方法に...
...,...,...,...,...,...,...,...,...,...,...
44559,2021.01.07,"三井住友建設株式会社,ファイベックス株式会社",特願2019-114410,"E01D 19/10 (2006.01),E01D 22/00 (2006.01)",E01D 19/10 (2006.01),"2D059AA21,2D059GG55,2D059GG40,2D059GG02","E01D 19/10 ,E01D 22/00 B",(57)【要約】【課題】コンクリートからなる壁高欄の下端部における内側及び外側の両面に引張部...,【特許請求の範囲】【請求項１】 コンクリート床版の側部が片持ち状に張り出した橋梁又は高架橋に...,【発明の詳細な説明】【技術分野】【０００１】 本発明は、コンクリート床版を有する橋梁、高架橋...
44560,2021.01.07,三井住友建設株式会社,特願2019-114061,"E04H 9/02 (2006.01),E04B 1/04 (2006....",E04H 9/02 (2006.01),"2E139AA01,2E139AB03,2E139BD24,2E139BD23,2E139B...","E04H 9/02 321C,E04B 1/04 M,E04H 9/02...",(57)【要約】【課題】壁のＰＣａ化に伴う問題を解消でき、且つ期待される減衰力を地震時に確実...,【特許請求の範囲】【請求項１】 建物の制振構造であって、 １対のＲＣ造の柱と、 上下に異なる...,【発明の詳細な説明】【技術分野】【０００１】 本開示は、建物の制振構造及びその構築方法に関し...
44561,2021.01.07,清水建設株式会社,特願2019-114049,"G01S 19/43 (2010.01),G01S 19/22 (2010....",G01S 19/43 (2010.01),"5J062AA09,5J062CC07,5J062EE04,5J062EE01,5J062DD23","G01S 19/43 ,G01S 19/22 ,G01S 19/28...",(57)【要約】【課題】測位アルゴリズムにおける最良の設定パラメータを迅速かつ確度良く決定す...,【特許請求の範囲】【請求項１】 複数の衛星からの衛星信号をもとに受信局の位置を計測する測位ア...,【発明の詳細な説明】【技術分野】【０００１】 本発明は、測位アルゴリズムにおける最良の設定パ...
44562,2021.01.07,株式会社フジタ,特願2019-113928,"C05G 3/40 (2020.01),C05B 17/00 (2006.01)",C05G 3/40 (2020.01),"4H061AA01,4H061BB29,4H061EE16,4H061LL26,4H061L...","C05G 3/00 103 ,C05B 17/00 ,C05G 3/40...",(57)【要約】【課題】作物に対する肥効成分を長期間利用可能な肥料を提供すること。【解決手段...,【特許請求の範囲】【請求項１】 リン及び鉄を担持する担体を含む肥料であって、 前記肥料中のり...,【発明の詳細な説明】【技術分野】【０００１】 本発明は肥料に関する。特に、本発明はリン及び鉄...


In [6]:
# df["company"] = df['出願人・権利者名'].str.split(",")    #出願名をスプリットしてset型に
# # df['datetime'] = [dt.strptime(i.split(',')[0],  '%Y.%m.%d') for i in df['公開・公表日']]  #公開・公表日を datetime64[ns] 型（ Timestamp 型）に 、複数あるのは先頭のだけに
# df['datetime'] = df['公開・公表日'].str.split(",").str[0]  #公開・公表日を datetime64[ns] 型（ Timestamp 型）に 、複数あるのは先頭のだけに
# # df['sentence'] = df['発明の名称'] +df['要約'] + df['全請求項'] .astype("str") + df['詳細な説明']  #なぜか .astype("str") いれないとエラー
# # df['sentence'] = df['発明の名称'] +df['要約'] + df['詳細な説明']  
# # df['sentence'] = df['発明の名称'] +df['要約']        #sentenceの対象にする項目も必要に応じて変える
# df['sentence'] = df['詳細な説明'] + df['要約']      #sentenceの対象にする項目も必要に応じて変える


In [7]:
df["company"] = df['出願人・権利者名'].str.replace(',', '|')
df['f_term'] = df['Ｆターム'].str.replace(',', '|')
df['head_IPC'] = df['筆頭ＩＰＣ'].str[:4]
df['year'] = df['公開・公表日'].str.split(",").str[0].str[:4]
df['sentence'] = df['詳細な説明'] + df['要約']

In [8]:
df.head(2)

,公開・公表日,出願人・権利者名,出願番号,IPC,筆頭ＩＰＣ,Ｆターム,FI,要約,全請求項,詳細な説明,company,f_term,head_IPC,year,sentence
0,2001.12.14,佐藤工業株式会社,実願2001-003729,B65D 5/491 (2006.01),B65D 5/491 (2006.01),"3E060AA03,3E060AB05,3E060BC02,3E060CB04,3E060C...","B65D 5/48 101J,B65D 5/498",【要約】【課題】 強度が劣る原紙を用いたとしても箱の強度が維持される外装用段ボール箱を得んと...,【実用新案登録請求の範囲】【請求項１】 広幅と狭幅の各一対の側板を交互に横方向に連設し、それ...,【考案の詳細な説明】 【０００１】 【考案の属する技術分野】 本考案は、袋菓子または詰...,佐藤工業株式会社,3E060AA03|3E060AB05|3E060BC02|3E060CB04|3E060C...,B65D,2001,【考案の詳細な説明】 【０００１】 【考案の属する技術分野】 本考案は、袋菓子または詰...
1,2001.11.16,"寄神建設株式会社,東亜建設工業株式会社,社団法人日本海上起重技術協会",実願2001-002834,B66C 1/36 (2006.01),B66C 1/36 (2006.01),"3F004CC01,3F004CD08",B66C 1/36 A,【要約】【課題】 重量物を吊り下げて設置を行う際の、重量物の開放を遠隔操作で容易かつ安全に行...,【実用新案登録請求の範囲】【請求項１】 一端がワイヤで吊り下げ支持される少なくとも一枚のフッ...,【考案の詳細な説明】【０００１】【考案の属する技術分野】 本考案は、クレーン等によりコンクリ...,寄神建設株式会社|東亜建設工業株式会社|社団法人日本海上起重技術協会,3F004CC01|3F004CD08,B66C,2001,【考案の詳細な説明】【０００１】【考案の属する技術分野】 本考案は、クレーン等によりコンクリ...


In [9]:
drop_list2 = ['出願人・権利者名', '公開・公表日', '要約', '全請求項', '詳細な説明', 'FI', '筆頭ＩＰＣ'
              , 'IPC', 'Ｆターム']
df = df.drop(drop_list2, axis=1)

df.head(2)


,出願番号,company,f_term,head_IPC,year,sentence
0,実願2001-003729,佐藤工業株式会社,3E060AA03|3E060AB05|3E060BC02|3E060CB04|3E060C...,B65D,2001,【考案の詳細な説明】 【０００１】 【考案の属する技術分野】 本考案は、袋菓子または詰...
1,実願2001-002834,寄神建設株式会社|東亜建設工業株式会社|社団法人日本海上起重技術協会,3F004CC01|3F004CD08,B66C,2001,【考案の詳細な説明】【０００１】【考案の属する技術分野】 本考案は、クレーン等によりコンクリ...


In [10]:
df['sentence'] = df['sentence'].astype(str)

In [11]:
excepword = ["位置","部","上記","上部","上端","下記","下部","下端","両端","内部","外部","前記","複数","横","図","要約","方法","手段","対象","現場","評価" ,"可能","構成","当該","以下","以上","該当","項","各","基準","端部","請求", "特許","本","一定","同上", '方向', '程度', '実施例', '実施'] # #除外する特定の名詞
exceptype= ["接尾","副詞可能","数詞","助数詞","代名詞","複数"]  #除外する名詞のタイプ


def tokenize_rm_setsubi(sentence):
    """ 連名詞のリストをトークンとして返す """
    # print(".",end='') #動いてるのか不安になるからプリント
    sentence = re.sub('\d+', '', sentence)   # 数字を削除(macabでなぜか名詞に認定された数字あったからここで)
    sentence = re.sub(r'[ -/:-@\[-~]', "。", sentence)#半角記号,数字,英字
    sentence = re.sub(r'[！-／：-＠［-｀｛-～、-〜”’%]+/g', "。", sentence)#全角記号
    sentence = re.sub(r'[[０-９ａ-ｚＡ-Ｚ]]', '', sentence) #全角英数を削除
    
    
    wakati = MeCab.Tagger(ipadic.MECAB_ARGS)
    node = wakati.parseToNode(sentence)
    sequence = 2   #何連名詞以上をかえすか
    dictionary = []
    prev_seq = 0  #その前にいくつ名詞続いたか
    flag = 0  #
    while node:
        word = node.surface
        hinshi = node.feature.split(",")

        if prev_seq  : # その前に名詞が1以上つづいていたら
            if  (hinshi[0] == "名詞" ) and  (hinshi[0] not in exceptype) and  (hinshi[1] not in exceptype )and (hinshi[2] not in exceptype )and (word not in excepword) : #今回、除外名詞以外だったら、
                dictionary[-1] = dictionary[-1]+word   # 辞書の最後の単語にこの単語を結合
                prev_seq  +=1
                flag = 0
                
            elif (hinshi[1] == "接尾" ): #接尾辞だったら
                dictionary[-1] = dictionary[-1]+word   # とりあえず辞書の最後の単語にこの単語を結合して
                prev_seq  +=1
                flag = 1 #フラグを1に

            elif flag == 1: # 一個前が接尾辞で今度が名詞でなかったら、一個前の接尾辞で終わってる単語を辞書から削除
                dictionary.pop(-1)   #一個前の接尾辞で終わってる単語を辞書から削除
                prev_seq  =0
                flag = 0
            else:
                flag = 0
                if prev_seq  < sequence: 
                    dictionary.pop(-1) #一個前の単語を辞書から削除
                prev_seq = 0
        else:  # その前に名詞がつづいてなくて
            if  (hinshi[0] == "名詞" or hinshi[0] == "接頭辞")  and  hinshi[0] not in exceptype and  hinshi[1] not in exceptype and hinshi[2] not in exceptype and word not in excepword :
                dictionary.append(word)
                prev_seq += 1
                flag = 0

        node = node.next  
    return dictionary

In [12]:
# 単語ー文書行列の作成
docs = np.array(df['sentence'])

tfidf_vectorizer = TfidfVectorizer(
    tokenizer = tokenize_rm_setsubi, #上でつくった関数
    lowercase=True,
    max_df=5000,
    # smooth_idf = False,
    min_df=5)

tfidf_vectorizer.fit(docs)
terms= tfidf_vectorizer.get_feature_names()

count_vectrizer = CountVectorizer(
    vocabulary = terms,
    tokenizer = tokenize_rm_setsubi, #上でつくった関数
)

c_matrix = count_vectrizer.transform(docs)

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
/tmp/ipykernel_103388/2207216377.py:11: FutureWarning: Possible nested set at position 1
  sentence = re.sub(r'[[０-９ａ-ｚＡ-Ｚ]]', '', sentence) #全角英数を削除


In [13]:
# np.save('c_matrix_allterm',c_matrix)
sp.sparse.save_npz('/app/data/c_matrix_allterm2021',c_matrix)
terms_df = pd.DataFrame(terms)
terms_df.to_csv('/app/data/terms_df2021.csv', mode='w')

# c_matrix = np.load('c_matrix_allterm.npz', allow_pickle=True)
# terms_df = pd.read_csv('terms_df.csv')


In [14]:
c_df = pd.DataFrame(terms_df.values, columns=["terms"] )
c_df["document_frequency"] =  np.array(c_matrix.astype(bool).sum(axis=0).reshape(-1,1))
c_df["max_term_frequency"]  = c_matrix.T.max(axis=1).toarray()
c_df["sumof_term_frequency"] = np.array(c_matrix.T.sum(axis=1))
c_df.sort_values('document_frequency', ascending=False) # 場合によってはコメントアウト

,terms,document_frequency,max_term_frequency,sumof_term_frequency
41005,断面形状,4782,54,13307
29716,外径,3992,133,14089
16440,作用効果,3925,22,7897
25298,回動,3319,126,19500
36379,所定間隔,3293,60,8118
...,...,...,...,...
39483,摩耗強度,5,4,10
60279,菌体増殖,5,1,5
60282,菌体膜,5,24,59
7913,バリアフリー構造,5,8,14


In [15]:
#単語:文書内単語頻度 の辞書の列つくって、pandasにくっつける
def makeTermsTfDict(terms, vec):
    # dic = {key: val for key, val in zip(terms, vec)  if val } #dfが1より大きいtermの辞書つくる
    # dic = {key: val for key, val in zip(terms, vec)  }
    # 20回以上登場する単語のみ抽出
    dic = {key: val for key, val in zip(terms, vec)  if val > 20 }
    term = ''
    for key in dic.keys():
        term += key + '|'
    # 最後の|を削除
    term = term[:-1]
    # dic2 = dict(sorted(dic.items(), key=lambda x:x[1],reverse=True)) #dfの値でソート
    return term

df['terms'] = pd.Series([makeTermsTfDict(terms, vec) for vec in [c_matrix.getrow(i).toarray()[0] for i in range(c_matrix.shape[0])]]) #countされたベクトルの各行を関数に入れていってシリーズに

df.to_excel('/app/data/patent2023ver.xlsx')
# df.to_json('parcedPatents.json')

In [16]:
df.head(10)

,出願番号,company,f_term,head_IPC,year,sentence,terms
0,実願2001-003729,佐藤工業株式会社,3E060AA03|3E060AB05|3E060BC02|3E060CB04|3E060C...,B65D,2001,【考案の詳細な説明】 【０００１】 【考案の属する技術分野】 本考案は、袋菓子または詰...,
1,実願2001-002834,寄神建設株式会社|東亜建設工業株式会社|社団法人日本海上起重技術協会,3F004CC01|3F004CD08,B66C,2001,【考案の詳細な説明】【０００１】【考案の属する技術分野】 本考案は、クレーン等によりコンクリ...,
2,特願2001-120638,株式会社大林組,2D059AA03|2D059CC04,E01D,2001,【発明の詳細な説明】【０００１】【産業上の利用分野】この発明は、橋脚柱の構築工法および橋脚柱...,
3,特願2001-109004,株式会社大林組,3F040BA01|3F040CA04|3F040EA03,E02B,2001,【発明の詳細な説明】【０００１】【発明の属する技術分野】この発明は、コンクリートの運搬装置に...,
4,特願2001-108995,株式会社大林組,NaN,E02B,2001,【発明の詳細な説明】【０００１】【発明の属する技術分野】この発明は、コンクリートの運搬方法に...,
5,特願2001-108994,株式会社大林組,3F040BA01|3F040EA03|3F040FA03|3F040FA05,E02B,2001,【発明の詳細な説明】【０００１】【発明の属する技術分野】この発明は、ダム用コンクリートの運搬...,
6,特願2001-108736,株式会社大林組,3F011AA02|3F011BA01|3F011BC07,E02B,2001,【発明の詳細な説明】【０００１】【発明の属する技術分野】この発明は、コンクリートの運搬方法に...,
7,特願2001-092130,大阪瓦斯株式会社|株式会社竹中工務店|株式会社竹中土木|三谷セキサン株式会社,2D041AA02|2D041BA13|2D041BA19|2D041CA03|2D041C...,E02D,2001,【発明の詳細な説明】【０００１】【発明の属する技術分野】この発明は、コンクリート杭と鋼管杭と...,コンクリート杭|肉厚|連結継手|鋼管杭
8,特願2001-085589,株式会社長谷工コーポレーション,NaN,E04H,2001,【発明の詳細な説明】【０００１】【発明の属する技術の分野】本発明は、マンション等、集合住宅の...,
9,特願2001-085263,株式会社長谷工コーポレーション,NaN,E04H,2001,【発明の詳細な説明】【０００１】【発明の属する技術の分野】本発明は、マンション等、集合住宅の...,


In [17]:
df.T.to_json('/app/data/patent2021ver.json', force_ascii=False, indent=4)

In [18]:
import pickle
with open('/app/data/patent_df2021.pkl', 'wb') as f:
    pickle.dump(df, f)